In [1]:
import numpy as np
from scipy import optimize
from scipy.optimize import linprog
from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint

### Exercise 1
# minimize
#    F = -4x[0] + 2x[1]

# subject to:
#     -5x[0] + 1x[1] <= 6
#       x[0] + 5x[1] <= 7
#               x[1] >= -3
#       -inf <= x[0] <= inf


c = [-4, 2]
A = [[-5, 1],[1, 5]]
b = [6, 7]
x0_bounds = (None, None)
x1_bounds = (-3, None)
res = linprog(c, A_ub=A, b_ub=b, bounds=(x0_bounds, x1_bounds), options={"disp": True})
print(res)


### Exercise 2
# minimize
#     F = x[1]^2 + 4x[2]^2 -32x[2] + 64

# subject to:
#      x[1] + x[2] <= 7      
#     -x[1] + 2x[2] <= 4     
#      x[1] +  0*x[2] >= 0   -> -x[1] + 0    <= 0
#     0*x[1] + x[2] >= 0     ->   0  + -x[2] <= 0
#     0*x[1] +x[2] <= 4      ->  0   + x[2]  <= 4

# in matrix notation:
#     F = (1/2)*x.T*H*x + c*x + c0

# subject to:
#     Ax <= b   ----use-other-form--->    0 <= b - Ax

#function:
H = np.array([1*2,4*2]) # qadratic term

c = np.array([0,-32]) #linear term

c0 = 64     # const term

#constraints
A = np.array([[1,1],[-1,2],[-1,0],[0,-1],[0,1]]) # linear constrain-term

b = np.array([7,4,0,0,4])          # constant constrain-term

x0 = np.random.randn(2)

def F(x):
    #     F = (1/2)*x.T*H*x + c*x + c0
    return .5*np.dot(x,H*x) + np.dot(c,x) + c0
    #return x[0]**2 + 4*x[1]**2 -32*x[1] + 64

def jac(x):
    #return np.array([2*x[0], 8*x[1]-32])
    return np.array(H*x + c)

#bounds = Bounds([0, None], [0, 4.0])

cons = {'type': 'ineq',
            'fun' : lambda x: np.array(b-np.dot(A,x)),
            'jac' : lambda x: np.array(-A)}

cons_no_jac = {'type': 'ineq',
            'fun' : lambda x: np.array(b-np.dot(A,x))}

solver = 'SLSQP'

def opt_const_jac():
    return optimize.minimize(F, x0, jac=jac,constraints=cons, method=solver)

def opt_const_no_jac():
    return optimize.minimize(F, x0, constraints=cons_no_jac, method=solver)

def opt_no_const_jac():
    return optimize.minimize(F, x0, jac=jac, method=solver,)

def opt_no_const_no_jac():
    return optimize.minimize(F, x0, method=solver)

Optimization terminated successfully.
         Current function value: -94.000000  
         Iterations: 4
     con: array([], dtype=float64)
     fun: -94.0
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([119.,   0.])
  status: 0
 success: True
       x: array([22., -3.])


In [2]:
# implement timeit method
import timeit
print("-------opt_const_jac()----------")
%timeit opt_const_jac()
print("\n-------opt_const_no_jac()----------")
%timeit opt_const_no_jac()
print("\n-------opt_no_const_jac()----------")
%timeit opt_no_const_jac()
print("\n-------opt_no_const_no_jac()----------")
%timeit opt_no_const_no_jac()

-------opt_const_jac()----------
443 µs ± 27.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

-------opt_const_no_jac()----------
703 µs ± 4.12 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

-------opt_no_const_jac()----------
274 µs ± 10.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

-------opt_no_const_no_jac()----------
382 µs ± 959 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
